## Extract facts about tJim Parsons’s wikipedia pages

We'll start with the title search string 'Jim Parsons' to retrieve the page about Jim Parsons.

In [4]:
import requests
title='Jim Parsons'
response = requests.get("https://simple.wikipedia.org/w/api.php?format=json&action=query&titles="+str(title)+"&prop=revisions&rvprop=content")
response

<Response [200]>

The response object is an HTTP GET response. It turns out the requests package contains a json interpreter, which we can invoke as:

In [5]:
jsondata = response.json()

a couple of helper routines:
The first routine converts the JSON to a carefully-formatted string. The second writes a string to a file. We can use them together to save the JSON data to a better format for viewing.

In [6]:
import json

def pretty(jdata):    
    str = json.dumps(jdata, sort_keys=True, indent=4).decode('string_escape')    
    return str

def saveas(sdata, fname):    
    f = open(fname,'w')    
    f.write(sdata)
    f.close()

In [7]:
saveas(pretty(jsondata), title+'.json')

In [8]:
content = jsondata['query']['pages'].values()[0]['revisions'][-1]
content = content.get('*')

In [9]:
import mwparserfromhell as mwph 
wikicode = mwph.parse(content)

we want to parse the english text from the article, we want to ignore all these metadata. MWPH has a method to do this:

In [10]:
text = wikicode.strip_code()

In [11]:
saveas(pretty(text), title+'_simple.txt')

In [12]:
from lxml import etree 
parser = etree.XMLParser(recover=True)
tree = etree.parse('Jim Parsons.xml',parser)

In [13]:
root=tree.getroot() 

In [14]:
title = 'Parsons'
def printnode(node):    
    for i in node.findall(".//leaf"):        
        print(" " + i.attrib['value']),    
    print('') 

def testnode(node, agent, action):    
    aa = node.findall("./node[@value='NP']//node[@value='NNP']//leaf[@value='"+agent+"']")    
    bb = node.findall("./node[@value='VP']//node[@value='VBZ']//leaf[@value='"+action+"']")    
    if (len(aa) > 0 and len(bb) > 0):        
        printnode(node)     

def agentact(node, agent, action):    
    testnode(node, agent, action)    
    snodes = node.findall(".//node[@value='S']")    
    for snode in snodes:        
        testnode(snode, agent, action)


In [15]:
map(lambda (nn): agentact(nn[0][0][0], title, 'is'), root)
[]

 ``  James  Joseph  ``  Jim  ''  Parsons  -LRB-  born  March  24  ,  1973  in  Houston  ,  Texas  -RRB-  is  an  American  actor  . 
 Parsons  is  best  known  for  playing  the  role  of  Sheldon  Cooper  in  the  television  series  ,  The  Big  Bang  Theory  . 
 According  to  The  New  York  Times  ,  Parsons  is  gay.Stalked  by  shadows  Filmography  Movies  Nowhere  to  Go 


[]